In [ ]:
using Distances, LinearAlgebra, Random, Statistics, Distributions
include("KernelQuantile.jl")
include("KernelUtils.jl")
include("KernelLogistic.jl")
include("KernelMultinomial.jl")

In [ ]:
p = 50
n = 16384
covM = ones(p,p)
for i in 1:p
    for j in 1:p
        covM[i,j] = 0.5^abs(i-j)
    end
end
λs = exp10.(range(1, -5, length=30))
ms = Int.(2 .^(range(3, 12, length=10)))
nreps = 10

In [ ]:
quantile_errors1 = zeros(nreps,length(ms))
quantile_times_MM1 = zeros(nreps,length(ms))
quantile_times_newton1 = zeros(nreps,length(ms))
quantile_errors_linear = zeros(nreps)
quantile_errors2 = zeros(nreps,length(ms))
quantile_times_MM2 = zeros(nreps,length(ms))
quantile_times_newton2 = zeros(nreps,length(ms))
for i in 1:nreps
  d = MvNormal(zeros(p), covM)
  X = Matrix(Transpose(rand(d,n)))
  d2 = TDist(1.5)
  ηs = -4 .+ sin.(X[:,1]).+ X[:,2] .* X[:, 3]  .+ X[:,4].^3 .- abs.(X[:,5]) .+0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
  y = ηs .+ rand(d2, n)
  X_test = Matrix(Transpose(rand(d,Int(n/4))))
  ηs_test = -4 .+ sin.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 3] .+ X_test[:,4].^3 .- abs.(X_test[:,5]) .+ 0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
  y_test = ηs_test .+ rand(d2, Int(n/4))
  σ = 10.0
  K = rbf_kernel(X,σ)
  for j in 1:length(ms)
     m = ms[j]
     G = generate_nystrom_G(n,m)
     @time βs1,total_time1 = kernel_quantile(y,K,G,0.5,λs,h=0.25,verbose=false,ϵ=1e-9)
     @time βs2,total_time2 = kernel_quantile_newton(y,K,G,0.5,λs,h=0.25,verbose=false,ϵ=1e-9)
     best_metric = Inf
     for k in 1:length(λs)
        best_metric = min(mean(abs.(ηs_test .- predict_krr(X,X_test,G,βs1[k,:],σ))),best_metric)
     end
     quantile_errors1[i,j] = best_metric
     quantile_times_MM1[i,j] = total_time1
     quantile_times_newton1[i,j] = total_time2
  end
  σ = 20.0
  K = rbf_kernel(X,σ)
  for j in 1:length(ms)
     m = ms[j]
     G = generate_nystrom_G(n,m)
     @time βs1,total_time1 = kernel_quantile(y,K,G,0.5,λs,h=0.25,verbose=false,ϵ=1e-9)
     @time βs2,total_time2 = kernel_quantile_newton(y,K,G,0.5,λs,h=0.25,verbose=false,ϵ=1e-9)
     best_metric = Inf
     for k in 1:length(λs)
        best_metric = min(mean(abs.(ηs_test .- predict_krr(X,X_test,G,βs1[k,:],σ))),best_metric)
     end
     quantile_errors2[i,j] = best_metric
     quantile_times_MM2[i,j] = total_time1
     quantile_times_newton2[i,j] = total_time2
  end
  β3, _ , _, _ = FastQR([ones(size(X,1)) X], y, 0.5, kernel="Gaussian")
  quantile_errors_linear[i] = mean(abs.(ηs_test .- [ones(size(X_test,1)) X_test]*β3))
end

In [ ]:
using DelimitedFiles
writedlm("quantile_times_MM1.csv",quantile_times_MM1,",")
writedlm("quantile_times_newton1.csv",quantile_times_newton1,",")
writedlm("quantile_times_MM2.csv",quantile_times_MM2,",")
writedlm("quantile_times_newton2.csv",quantile_times_newton2,",")
writedlm("quantile_errors1.csv",quantile_errors1,",")
writedlm("quantile_errors2.csv",quantile_errors2,",")
writedlm("quantile_errors_linear.csv",quantile_errors_linear,",")

In [ ]:
logistic_errors1 = zeros(nreps,length(ms))
logistic_errors2 = zeros(nreps,length(ms))
logistic_times_MM1 = zeros(nreps,length(ms))
logistic_times_MM2 = zeros(nreps,length(ms))
logistic_times_newton1 = zeros(nreps,length(ms))
logistic_times_newton2 = zeros(nreps,length(ms))
logistic_errors_linear = zeros(nreps)
for i in 1:nreps
  d = MvNormal(zeros(p), covM)
  X = Matrix(Transpose(rand(d,n)))
  ηs = -4 .+ sin.(X[:,1]).+ X[:,2] .* X[:, 3] .+  .+ X[:,4].^3 .- abs.(X[:,5]) .+ 0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
  X_test = Matrix(Transpose(rand(d,Int(n/4))))
  ηs_test = -4 .+ sin.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 3] .+ X_test[:,4].^3 .- abs.(X_test[:,5]) .+ 0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
  y = zeros(n)
  for i = 1:n
    u = 1/(1+exp(-ηs[i]))
    if rand() < u
         y[i] = 1.0 
    end
  end
  y_test = zeros(Int(n/4))
  for i = 1:Int(n/4)
    u = 1/(1+exp(-ηs_test[i]))
    if rand() < u
         y_test[i] = 1.0 
    end
  end
  σ = 15.0
  K = rbf_kernel(X,σ)
  for j in 1:length(ms)
     m = ms[j]
     G = generate_nystrom_G(n,m)
     @time βs1, total_time1 = kernel_logistic(y,K,G,λs,verbose=false,ϵ=1e-9)
     @time βs2, total_time2 = kernel_logistic_newton(y,K,G,λs,verbose=false,ϵ=1e-9)
     best_metric = -Inf
     for k in 1:length(λs)
        best_metric = max(loglikelihood(y_test, predict_krr(X,X_test,G,βs1[k,:],σ)),best_metric)
     end
     logistic_errors1[i,j] = best_metric
     logistic_times_MM1[i,j] = total_time1
     logistic_times_newton1[i,j] = total_time2
  end
  σ = 30.0
  K = rbf_kernel(X,σ)
  for j in 1:length(ms)
     m = ms[j]
     G = generate_nystrom_G(n,m)
     @time βs1, total_time1 = kernel_logistic(y,K,G,λs,verbose=false,ϵ=1e-9)
     @time βs2, total_time2 = kernel_logistic_newton(y,K,G,λs,verbose=false,ϵ=1e-9)
     best_metric = -Inf
     for k in 1:length(λs)
        best_metric = max(loglikelihood(y_test, predict_krr(X,X_test,G,βs1[k,:],σ)),best_metric)
     end
     logistic_errors2[i,j] = best_metric
     logistic_times_MM2[i,j] = total_time1
     logistic_times_newton2[i,j] = total_time2
  end
  β3, _ , _ = FastLogistic([ones(size(X,1)) X], y)
  logistic_errors_linear[i] = loglikelihood(y_test, [ones(size(X_test,1)) X_test]*β3)
end

In [ ]:
using DelimitedFiles
writedlm("logistic_times_MM1.csv",logistic_times_MM1,",")
writedlm("logistic_times_newton1.csv",logistic_times_newton1,",")
writedlm("logistic_times_MM2.csv",logistic_times_MM2,",")
writedlm("logistic_times_newton2.csv",logistic_times_newton2,",")
writedlm("logistic_errors1.csv",logistic_errors1,",")
writedlm("logistic_errors2.csv",logistic_errors2,",")
writedlm("logistic_errors_linear.csv",logistic_errors_linear,",")

In [ ]:
nreps = 10
ms = Int.(2 .^(range(2, 11, length=10)))
multinomial_errors1 = zeros(nreps,length(ms))
multinomial_errors2 = zeros(nreps,length(ms))
multinomial_errors_linear = zeros(nreps)
multinomial_times_MM1 = zeros(nreps,length(ms))
multinomial_times_newton1 = zeros(nreps,length(ms))
multinomial_times_MM2 = zeros(nreps,length(ms))
multinomial_times_newton2 = zeros(nreps,length(ms))
for i in 1:nreps
  d = MvNormal(zeros(p), covM)
  X = Matrix(Transpose(rand(d,n)))
  ηs1 = -4 .+ sin.(X[:,1]).+ X[:,2] .* X[:, 3]  .+ X[:,4].^3 .- abs.(X[:,5]) .+0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
  ηs2 = 4 .+ cos.(X[:,1]).+ X[:,2] .* X[:, 4]  .+ X[:,5].^3 .- abs.(X[:,3]) .-0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
  ηs = [ηs1 ηs2]
  row_sums_buffer = zeros(n)
  P = zeros(n,3)
  P_pred = zeros(Int(n/4),3)
  compute_probs_reduced!(P,ηs, row_sums_buffer)
  Y = zeros(n,3)
  for k in 1:n
    probs = P[k, :]
    d3 = Multinomial(1, probs)
    Y[k,:] .= rand(d3)
  end
  X_test = Matrix(Transpose(rand(d,Int(n/4))))
  ηs1_test = -4 .+ sin.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 3]  .+ X_test[:,4].^3 .- abs.(X_test[:,5]) .+0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
  ηs2_test = 4 .+ cos.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 4]  .+ X_test[:,5].^3 .- abs.(X_test[:,3]) .-0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
  ηs_test = [ηs1_test ηs2_test]
  P = zeros(Int(n/4),3)
  compute_probs_reduced!(P,ηs_test, row_sums_buffer)
  Y_test = zeros(Int(n/4),3)
  for k in 1:Int(n/4)
    probs = P[k, :]
    d3 = Multinomial(1, probs)
    Y_test[k,:] .= rand(d3)
  end
  σ = 15.0
  K = rbf_kernel(X,σ)
  for j in 1:length(ms)
     m = ms[j]
     G = generate_nystrom_G(n,m)
     @time B1s, total_time1 = KernelMultinomial(Y,K,G,λs,ϵ=1e-4,verbose=false)
     @time B2s, total_time2 = KernelMultinomial_newton(Y,K,G,λs,ϵ=1e-4,verbose=false)
     best_metric = -Inf
     for k in 1:length(λs)
        compute_probs_reduced!(P_pred,predict_krr(X,X_test,G,B1s[k,:,:],σ), row_sums_buffer)
        best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
     end
     println(i)
     println(j)
     multinomial_errors1[i,j] = best_metric
     multinomial_times_MM1[i,j] = total_time1
     multinomial_times_newton1[i,j] = total_time2
  end
  σ = 30.0
  K = rbf_kernel(X,σ)
  for j in 1:length(ms)
     m = ms[j]
     G = generate_nystrom_G(n,m)
     @time B1s, total_time1 = KernelMultinomial(Y,K,G,λs,ϵ=1e-4,verbose=false)
     @time B2s, total_time2 = KernelMultinomial_newton(Y,K,G,λs,ϵ=1e-4,verbose=false)
     best_metric = -Inf
     for k in 1:length(λs)
        compute_probs_reduced!(P_pred,predict_krr(X,X_test,G,B1s[k,:,:],σ), row_sums_buffer)
        best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
     end
     multinomial_errors2[i,j] = best_metric
     multinomial_times_MM2[i,j] = total_time1
     multinomial_times_newton2[i,j] = total_time2
  end
  B_linear,_,_ = FastMultinomial_reduced([ones(size(X,1)) X],Y)
  P_pred = zeros(Int(n/4),3)
  compute_probs_reduced!(P_pred,[ones(size(X_test,1)) X_test]*B_linear, row_sums_buffer)
  multinomial_errors_linear[i] = loglikelihood(Y_test, P_pred)
end

In [ ]:
using DelimitedFiles
writedlm("multinomial_times_MM1.csv",multinomial_times_MM1,",")
writedlm("multinomial_times_newton1.csv",multinomial_times_newton1,",")
writedlm("multinomial_times_MM2.csv",multinomial_times_MM2,",")
writedlm("multinomial_times_newton2.csv",multinomial_times_newton2,",")
writedlm("multinomial_errors1.csv",multinomial_errors1,",")
writedlm("multinomial_errors2.csv",multinomial_errors2,",")
writedlm("multinomial_errors_linear.csv",multinomial_errors_linear,",")

In [ ]:
using DelimitedFiles
quantile_times_MM = convert(Matrix{Float64},readdlm("quantile_times_MM2.csv", ',', Any, '\n'))
quantile_times_newton = convert(Matrix{Float64},readdlm("quantile_times_newton2.csv", ',', Any, '\n'))
quantile_errors = convert(Matrix{Float64},readdlm("quantile_errors2.csv", ',', Any, '\n'))
quantile_errors_linear = convert(Matrix{Float64},readdlm("quantile_errors_linear.csv", ',', Any, '\n'))   
logistic_times_MM = convert(Matrix{Float64},readdlm("logistic_times_MM2.csv", ',', Any, '\n'))
logistic_times_newton = convert(Matrix{Float64},readdlm("logistic_times_newton2.csv", ',', Any, '\n'))
logistic_errors = convert(Matrix{Float64},readdlm("logistic_errors2.csv", ',', Any, '\n'))
logistic_errors_linear = convert(Matrix{Float64},readdlm("logistic_errors_linear.csv", ',', Any, '\n'))  
multinomial_times_MM = convert(Matrix{Float64},readdlm("multinomial_times_MM2.csv", ',', Any, '\n'))
multinomial_times_newton = convert(Matrix{Float64},readdlm("multinomial_times_newton2.csv", ',', Any, '\n'))
multinomial_errors = convert(Matrix{Float64},readdlm("multinomial_errors2.csv", ',', Any, '\n'))
multinomial_errors_linear = convert(Matrix{Float64},readdlm("multinomial_errors_linear.csv", ',', Any, '\n'))  

In [ ]:
using Plots
using Measures
using Statistics

# Example data for demonstration (replace with your actual data)
log2m = 3:12
# Create a 3x1 plot layout
plot(layout = (2,3),size=(1200,800), margin=5mm)

plot!(log2m, mean(quantile_times_MM,dims=1)[:], yerr = std(quantile_times_MM,dims=1)[:], label = "Quadratic MM", xlabel = "log2(m)", ylabel = "Time (seconds)", title = "Quantile Regression",subplot=1,color="red",markerstrokecolor=:red)
plot!(log2m, mean(quantile_times_newton,dims=1)[:], yerr = std(quantile_times_newton,dims=1)[:], label = "Newton", subplot=1,color="blue",markerstrokecolor=:blue)

plot!(log2m, mean(logistic_times_MM,dims=1)[:], yerr = std(logistic_times_MM,dims=1)[:], label = "Quadratic MM", xlabel = "log2(m)", ylabel = "Time (seconds)",title = "Logistic Regression",subplot=2,color="red",markerstrokecolor=:red)
plot!(log2m, mean(logistic_times_newton,dims=1)[:], yerr = std(logistic_times_newton,dims=1)[:], label = "Newton", subplot=2,color="blue",markerstrokecolor=:blue, legend=:none)

plot!(2:11, mean(multinomial_times_MM,dims=1)[:], yerr = std(multinomial_times_MM,dims=1)[:], label = "Quadratic MM", xlabel = "log2(m)", ylabel = "Time (seconds)", title = "Multinomial Regression",subplot=3,color="red",markerstrokecolor=:red)
plot!(2:11, mean(multinomial_times_newton,dims=1)[:], yerr = std(multinomial_times_newton,dims=1)[:], label = "Newton", subplot=3,color="blue",markerstrokecolor=:blue, legend=:none)

plot!(log2m, mean(quantile_errors,dims=1)[:], yerr = std(quantile_errors,dims=1)[:], label = "Kernel", xlabel = "log2(m)", ylabel = "MAD",subplot=4,color="blue",markerstrokecolor=:blue)
hline!([mean(quantile_errors_linear)], label = "Linear", color="green",markerstrokecolor=:green, linestyle = :dash,subplot=4)

plot!(log2m, mean(logistic_errors,dims=1)[:], yerr = std(logistic_errors,dims=1)[:], label = "Kernel", xlabel = "log2(m)", ylabel = "Log-likelihood",subplot=5,color="blue",markerstrokecolor=:blue)
hline!([mean(logistic_errors_linear)], label = "Linear", color="green",markerstrokecolor=:green, linestyle = :dash,subplot=5,legend=:none)

plot!(2:11, mean(multinomial_errors,dims=1)[:], yerr = std(multinomial_errors,dims=1)[:], label = "Kernel", xlabel = "log2(m)", ylabel = "Log-likelihood",subplot=6,color="blue",markerstrokecolor=:blue)
hline!([mean(multinomial_errors_linear)], label = "Linear", color="green",markerstrokecolor=:green, linestyle = :dash,subplot=6,legend=:none)

savefig("sigma30.pdf")